# ENSO

This notebook provides an overview of running the modes of variability module. More information can be found in the [README](https://github.com/PCMDI/pcmdi_metrics/tree/master/pcmdi_metrics/variability_mode). Example parameter files are located in the [PMP sample setups](https://github.com/PCMDI/pcmdi_metrics/tree/master/sample_setups/pcmdi_parameter_files/variability_modes). 

It is expected that the user has run the Demo_0_download_data notebook to obtain the sample data and generate parameter files. This first cell loads the demo directory choices made in Demo_0_download_data.

## Download demo data

In [1]:
# Lets get the file containing the data needed for this demo
import requests
r = requests.get("https://pcmdiweb.llnl.gov/pss/pmpdata/pmp_enso_tutorial_files.txt")
with open("enso_data_files.txt","wb") as f:
    f.write(r.content)

In [2]:
# This is where you will be downloading the sample_data
demo_data_directory = "demo_data"
# this line is where your output will be stored
demo_output_directory = "deom_output"

In [3]:
# Let's download the files
import cdat_info
try:
    cdat_info.download_sample_data_files("enso_data_files.txt", demo_data_directory)
    print("All files downloaded")
except:
    print("Download failed")

All files downloaded


In [4]:
from download_sample_data import generate_parameter_files
filenames=["basic_enso_param.py.in"]
generate_parameter_files(demo_data_directory, demo_output_directory, filenames=filenames)

Preparing parameter file: basic_enso_param.py
Saving User Choices


## Environment

[ENSO Metrics package](https://github.com/CLIVAR-PRP/ENSO_metrics) and [scipy](https://www.scipy.org/) installation needed

In [5]:
"""
%%bash

git clone https://github.com/CLIVAR-PRP/ENSO_metrics.git
cd ENSO_metrics
python setup.py install
"""

"""
conda install -c anaconda scipy 
"""

'\nconda install -c anaconda scipy \n'

## Usage

ENSO can be run on the command line or with using a parameter file

For help, type:  
```
enso_driver.py --help
```

In [6]:
%%bash
enso_driver.py --help

usage: enso_driver.py [-h] [--parameters PARAMETERS]
                      [--diags OTHER_PARAMETERS [OTHER_PARAMETERS ...]]
                      [--mip MIP] [--exp EXP] [--modpath MODPATH]
                      [--modpath_lf MODPATH_LF]
                      [--modnames MODNAMES [MODNAMES ...]] [-r REALIZATION]
                      [--reference_data_path REFERENCE_DATA_PATH]
                      [--reference_data_lf_path REFERENCE_DATA_LF_PATH]
                      [--metricsCollection METRICSCOLLECTION]
                      [--json_name JSON_NAME] [--netcdf_name NETCDF_NAME]
                      [--results_dir RESULTS_DIR] [--case_id CASE_ID]
                      [--obs_catalogue OBS_CATALOGUE]
                      [--obs_cmor_path OBS_CMOR_PATH] [-d [DEBUG]]
                      [--obs_cmor [OBS_CMOR]] [--nc_out [NC_OUT]]

optional arguments:
  -h, --help            show this help message and exit
  --parameters PARAMETERS, -p PARAMETERS
  --diags OTHER_PARAMETERS [OTHER_PA

## Basic example

In [7]:
%%time
%%bash
enso_driver.py -p basic_enso_param.py

mip: cmip5
exp: historical
models: ['ACCESS1-0']
realization:  r1i1p1
mc_name: ENSO_perf
outdir: deom_output/basicTestEnso/ENSO_perf
netcdf_path: deom_output/basicTestEnso/ENSO_perf
debug: False
obs_cmor: True
obs_cmor_path: demo_data/PCMDIobs2
egg_pth: /Users/lee1043/Library/Caches/Python-Eggs/pcmdi_metrics-v1.2.1_691_g26ef287-py3.8.egg-tmp/share/pmp
output directory for graphics:deom_output/basicTestEnso/ENSO_perf
output directory for diagnostic_results:deom_output/basicTestEnso/ENSO_perf
output directory for metrics_results:deom_output/basicTestEnso/ENSO_perf
list_variables: ['pr', 'sst', 'taux']
list_obs: ['AVISO-1-0', 'ERA-INT', 'GPCP-2-3', 'HadISST-1-1']
PMPdriver: dict_obs readin end
models: ['ACCESS1-0']
 ----- model:  ACCESS1-0  ---------------------
PMPdriver: var loop start for model  ACCESS1-0
realization: r1i1p1
 --- run:  r1i1p1  ---
 --- var:  pr  ---
var_in_file: pr
var, areacell_in_file, realm: pr areacella atmos
path:  demo_data/CMIP5_demo_data/pr_Amon_ACCESS1-0_histo

/opt/anaconda3/envs/pmp_nightly_20210602/lib/python3.8/site-packages/cdms2/MV2.py:318: Warning: arguments order for compress function has changed
it is now: MV2.copmress(array,condition), if your code seems to not react or act wrong to a call to compress, please check this
  warnings.warn(
INFO::2021-06-04 11:34::pcmdi_metrics:: Results saved to a json file: /Users/lee1043/Documents/Research/git/pcmdi_metrics/doc/jupyter/Demo/deom_output/basicTestEnso/ENSO_perf/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1.json
INFO::2021-06-04 11:34::pcmdi_metrics:: Results saved to a json file: /Users/lee1043/Documents/Research/git/pcmdi_metrics/doc/jupyter/Demo/deom_output/basicTestEnso/ENSO_perf/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1_diveDown.json


CPU times: user 8.14 ms, sys: 7.34 ms, total: 15.5 ms
Wall time: 3min 44s


In [8]:
!cat deom_output/basicTestEnso/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1.json

{
    "DISCLAIMER": "USER-NOTICE: The results in this file were produced with the PMP v1.1 (https://github.com/PCMDI/pcmdi_metrics). They are for research purposes only. They are subject to ongoing quality control and change as the PMP software advances, interpolation methods are modified, observational data sets are updated, problems with model data are corrected, etc. Use of these results for research (presentation, publications, etc.) should reference: Gleckler, P. J., C. Doutriaux, P. J. Durack, K. E. Taylor, Y. Zhang, and D. N. Williams, E. Mason, and J. Servonnat (2016), A more powerful reality test for climate models, Eos, 97, doi:10.1029/2016EO051663. If any problems are uncovered in using these results please contact the PMP development team at pcmdi-metrics@llnl.gov\n",
    "REFERENCE": "MC for ENSO Performance...",
    "RESULTS": {
        "model": {
            "ACCESS1-0": {
                "r1i1p1": {
                    "metadata": {
                        "description_

In [9]:
%%time
%%bash
enso_driver.py -p basic_enso_param.py --metricsCollection ENSO_tel

mip: cmip5
exp: historical
models: ['ACCESS1-0']
realization:  r1i1p1
mc_name: ENSO_tel
outdir: deom_output/basicTestEnso/ENSO_perf
netcdf_path: deom_output/basicTestEnso/ENSO_perf
debug: False
obs_cmor: True
obs_cmor_path: demo_data/PCMDIobs2
egg_pth: /Users/lee1043/Library/Caches/Python-Eggs/pcmdi_metrics-v1.2.1_691_g26ef287-py3.8.egg-tmp/share/pmp
output directory for graphics:deom_output/basicTestEnso/ENSO_perf
output directory for diagnostic_results:deom_output/basicTestEnso/ENSO_perf
output directory for metrics_results:deom_output/basicTestEnso/ENSO_perf
list_variables: ['pr', 'sst']
list_obs: ['AVISO-1-0', 'ERA-INT', 'GPCP-2-3', 'HadISST-1-1']
PMPdriver: dict_obs readin end
models: ['ACCESS1-0']
 ----- model:  ACCESS1-0  ---------------------
PMPdriver: var loop start for model  ACCESS1-0
realization: r1i1p1
 --- run:  r1i1p1  ---
 --- var:  pr  ---
var_in_file: pr
var, areacell_in_file, realm: pr areacella atmos
path:  demo_data/CMIP5_demo_data/pr_Amon_ACCESS1-0_historical_r1i

INFO::2021-06-04 11:37::pcmdi_metrics:: Results saved to a json file: /Users/lee1043/Documents/Research/git/pcmdi_metrics/doc/jupyter/Demo/deom_output/basicTestEnso/ENSO_perf/cmip5_historical_ENSO_tel_basicTestEnso_ACCESS1-0_r1i1p1.json
INFO::2021-06-04 11:37::pcmdi_metrics:: Results saved to a json file: /Users/lee1043/Documents/Research/git/pcmdi_metrics/doc/jupyter/Demo/deom_output/basicTestEnso/ENSO_perf/cmip5_historical_ENSO_tel_basicTestEnso_ACCESS1-0_r1i1p1_diveDown.json


CPU times: user 7.84 ms, sys: 7.37 ms, total: 15.2 ms
Wall time: 2min 59s


In [10]:
%%time
%%bash
enso_driver.py -p basic_enso_param.py --metricsCollection ENSO_proc

mip: cmip5
exp: historical
models: ['ACCESS1-0']
realization:  r1i1p1
mc_name: ENSO_proc
outdir: deom_output/basicTestEnso/ENSO_perf
netcdf_path: deom_output/basicTestEnso/ENSO_perf
debug: False
obs_cmor: True
obs_cmor_path: demo_data/PCMDIobs2
egg_pth: /Users/lee1043/Library/Caches/Python-Eggs/pcmdi_metrics-v1.2.1_691_g26ef287-py3.8.egg-tmp/share/pmp
output directory for graphics:deom_output/basicTestEnso/ENSO_perf
output directory for diagnostic_results:deom_output/basicTestEnso/ENSO_perf
output directory for metrics_results:deom_output/basicTestEnso/ENSO_perf
list_variables: ['ssh', 'sst', 'taux', 'thf']
list_obs: ['AVISO-1-0', 'ERA-INT', 'GPCP-2-3', 'HadISST-1-1']
Observation datasetAVISO-1-0 is not given for variable thf
Observation datasetGPCP-2-3 is not given for variable thf
Observation datasetHadISST-1-1 is not given for variable thf
PMPdriver: dict_obs readin end
models: ['ACCESS1-0']
 ----- model:  ACCESS1-0  ---------------------
PMPdriver: var loop start for model  ACCESS1

/opt/anaconda3/envs/pmp_nightly_20210602/lib/python3.8/site-packages/cdms2/MV2.py:318: Warning: arguments order for compress function has changed
it is now: MV2.copmress(array,condition), if your code seems to not react or act wrong to a call to compress, please check this
  warnings.warn(
INFO::2021-06-04 11:39::pcmdi_metrics:: Results saved to a json file: /Users/lee1043/Documents/Research/git/pcmdi_metrics/doc/jupyter/Demo/deom_output/basicTestEnso/ENSO_perf/cmip5_historical_ENSO_proc_basicTestEnso_ACCESS1-0_r1i1p1.json
INFO::2021-06-04 11:39::pcmdi_metrics:: Results saved to a json file: /Users/lee1043/Documents/Research/git/pcmdi_metrics/doc/jupyter/Demo/deom_output/basicTestEnso/ENSO_perf/cmip5_historical_ENSO_proc_basicTestEnso_ACCESS1-0_r1i1p1_diveDown.json


CPU times: user 7.04 ms, sys: 6.55 ms, total: 13.6 ms
Wall time: 2min 33s
